# Sistema completo

In [3]:
import torch
print("¿GPU disponible?:", torch.cuda.is_available())
print("GPU actual:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Ninguna")


¿GPU disponible?: True
GPU actual: NVIDIA GeForce RTX 4070 Ti



###  **Importaciones del Notebook y sus Instalaciones**

Este notebook implementa diversas funcionalidades relacionadas con:

* Detección facial
* Procesamiento de video e imágenes
* Reconocimiento óptico de caracteres (OCR)
* Automatización web
* Verificación de identidad facial y deepfake detection

A continuación se explica cada una de las importaciones y su uso:

---

#### 🔸 **Importaciones utilizadas y explicación:**

##### Librerías estándar y generales:

* **`os`**: Manejo de archivos y rutas.
* **`time`**: Manejo de tiempos de espera y medición de tiempos.
* **`re`**: Expresiones regulares, para extracción de CURP y fechas.

##### Procesamiento de imágenes y video:

* **`cv2`** (`opencv-python`): Captura, procesamiento, visualización y guardado de imágenes y video.
* **`imutils`**: Herramienta que facilita tareas comunes con OpenCV (contornos, redimensionar imágenes).

##### Reconocimiento facial y detección facial avanzada:

* **`mediapipe`**: Para detección facial y extracción de landmarks (puntos clave del rostro).
* **`face_recognition`**: Comparación facial para validar identidad.

##### OCR (Reconocimiento Óptico de Caracteres):

* **`paddleocr`**: Reconocimiento de texto de la INE.

##### Automatización web:

* **`selenium`**: Automatización de consultas en páginas web (sitio del CURP).

##### Machine Learning (Deep Learning):

* **`torch`** (`PyTorch`): Librería para deep learning.
* **`torch.nn`**: Bloques fundamentales para construir redes neuronales.
* **`torchvision`**: Modelos preentrenados (EfficientNet) y transformación de imágenes.
* **`numpy`**: Manejo de arreglos numéricos y cálculos matriciales.


##### **Instalación completa de dependencias (`pip install`)**

A continuación, se presentan los comandos para instalar todas las librerías necesarias. Es recomendable hacerlo en un entorno virtual:

```bash
# Procesamiento de imágenes y video
pip install opencv-python imutils

# Detección y reconocimiento facial
pip install mediapipe face_recognition

# OCR (PaddleOCR)
pip install paddleocr paddlepaddle

# Automatización web (Selenium)
pip install selenium webdriver-manager

# Machine Learning (PyTorch, con GPU o CPU)
# Si tienes CUDA (GPU Nvidia):
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

# Si solo tienes CPU:
pip install torch torchvision torchaudio

# NumPy
pip install numpy
```

---

#### **Configuración recomendada del entorno virtual:**

Es altamente recomendable trabajar en un entorno virtual:

```bash
python -m venv .venv

# Activación del entorno virtual:
# Windows
.venv\Scripts\activate

# Linux/MacOS
source .venv/bin/activate
```

Después, instala todas las dependencias listadas anteriormente.


In [ ]:
# Librerías estándar
import os
import time
import re

# Procesamiento de imagen y video
import cv2
import imutils

# Automatización web (Selenium)
from selenium import webdriver
from selenium.common.exceptions import WebDriverException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# OCR con PaddleOCR
from paddleocr import PaddleOCR

# Detección y reconocimiento facial
import mediapipe as mp
import face_recognition

# Deep Learning (PyTorch y Torchvision)
import torch
import torch.nn as nn
import numpy as np
from torchvision import transforms, models


### Verificación de Vida con Detección Facial (MediaPipe)

Esta sección del sistema implementa un reto de vida en tiempo real, que solicita al usuario realizar gestos específicos (parpadeos, movimientos de cabeza) y verifica que la persona esté frente a la cámara, centrada, con ojos abiertos y sin movimiento brusco, como paso previo a la validación biométrica.

---

#### Inicialización de MediaPipe y configuración de índices

```python
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

LEFT_EYE_FULL = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_FULL = [362, 385, 387, 263, 373, 380]
```

* Se inicializa el módulo de malla facial de MediaPipe (`face_mesh`) y sus utilidades de dibujo (`drawing_utils`).
* Se definen los índices de los puntos clave del ojo izquierdo y derecho para calcular el parpadeo mediante Eye Aspect Ratio (EAR).

---

#### Función `eye_aspect_ratio()`

```python
def eye_aspect_ratio(landmarks, eye_indices, image_shape):
    ...
    return (A + B) / (2.0 * D)
```

* Calcula la relación de aspecto del ojo (EAR), que disminuye cuando el ojo se cierra.
* Se utiliza para detectar parpadeos reales.
* Toma como entrada los landmarks faciales detectados, los índices del ojo y el tamaño de la imagen.

---

#### Función `is_looking_forward()`

```python
def is_looking_forward(landmarks):
    ...
    return left and right
```

* Verifica si el usuario está mirando al frente, evaluando la posición de la pupila relativa a los bordes de los ojos.
* Esta información es clave para validar la toma de una selfie centrada con los ojos abiertos.

---

### Función principal `realizar_reto_de_vida()`

Esta es la función central del sistema de verificación facial en vivo.

#### Objetivo:

Verificar que el usuario:

1. Está físicamente presente.
2. Puede seguir instrucciones en tiempo real.
3. No está usando una reproducción (deepfake o imagen fija).
4. Está preparado para tomarse una selfie válida.

---

#### Etapas del reto de vida

1. **Captura de video en tiempo real**

   * Se activa la cámara web (`cv2.VideoCapture(0)`) y se guarda el video completo como `verificacion_video.mp4`.

2. **Detección facial con MediaPipe**

   * Se detectan landmarks faciales por frame.
   * Se dibuja la malla facial sobre el rostro usando `mp_drawing.draw_landmarks()`.

3. **Verificación de parpadeos (≥ 3)**

   * Se calcula el EAR y se detectan cierres y aperturas del ojo.
   * Se requiere que el usuario parpadee tres veces para superar esta etapa.

4. **Detección de movimientos de cabeza**

   * Asentir (sí): movimiento vertical del rostro (eje Y).
   * Negar (no): movimiento horizontal del rostro (eje X).
   * Se requiere 2 gestos válidos en cada caso.

5. **Transición a la fase selfie**

   * Al completar los gestos anteriores, el sistema cambia a la siguiente etapa.

---

#### Fase selfie automática

Una vez completado el reto de vida, el sistema espera que el usuario esté:

* Centrado: nariz dentro de un rango horizontal y vertical.
* Con ojos abiertos: EAR suficiente en ambos ojos.
* Estable: pocos cambios en los píxeles de los últimos frames.
* Mirando al frente: pupilas centradas.

Si todos los criterios se cumplen durante varios frames consecutivos, el sistema:

* Captura la imagen y la guarda como `selfie.jpg`.
* Finaliza exitosamente la función con `return True`.

---

#### Estructura del estado interno

* `blink_counter`, `nod_count`, `shake_count`: contadores de gestos realizados.
* `blink_completed`, `nod_completed`, `shake_completed`: estados de cada prueba.
* `fase_selfie`: bandera que indica si se está en la fase de captura de selfie.
* `gaze_start_time`, `stable_counter`: variables para validar centrado y estabilidad.

---

#### Resultado final

* Si todos los pasos fueron completados y se capturó la selfie, se devuelve `True`.
* En caso contrario (interrupción o falla en los gestos), se devuelve `False`.

---

Este sistema permite detectar actividad humana auténtica mediante gestos naturales y condiciones visuales, funcionando como una barrera efectiva contra suplantaciones por imagen o video.


In [ ]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

LEFT_EYE_FULL = [33, 160, 158, 133, 153, 144]
RIGHT_EYE_FULL = [362, 385, 387, 263, 373, 380]

def eye_aspect_ratio(landmarks, eye_indices, image_shape):
    def dist(p1, p2):
        x1, y1 = int(p1.x * image_shape[1]), int(p1.y * image_shape[0])
        x2, y2 = int(p2.x * image_shape[1]), int(p2.y * image_shape[0])
        return ((x2 - x1)**2 + (y2 - y1)**2) ** 0.5
    A = dist(landmarks[eye_indices[1]], landmarks[eye_indices[5]])
    B = dist(landmarks[eye_indices[2]], landmarks[eye_indices[4]])
    D = dist(landmarks[eye_indices[0]], landmarks[eye_indices[3]])
    return (A + B) / (2.0 * D)

def is_looking_forward(landmarks):
    def centered(pupil, outer, inner):
        d1 = abs(pupil - outer)
        d2 = abs(inner - pupil)
        ratio = d1 / (d1 + d2 + 1e-6)
        return 0.425 < ratio < 0.575
    left = centered(landmarks[468].x, landmarks[33].x, landmarks[133].x)
    right = centered(landmarks[473].x, landmarks[362].x, landmarks[263].x)
    return left and right

def realizar_reto_de_vida():
    cap = cv2.VideoCapture(0)
    frame_width, frame_height = int(cap.get(3)), int(cap.get(4))
    out = cv2.VideoWriter("verificacion_video.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 20, (frame_width, frame_height))
    face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

    # Estado
    blink_counter, blink_ready = 0, True
    blink_start_time, blink_completed = None, False
    nod_count, nod_stage, nod_start_y, nod_start_time, nod_completed = 0, "neutral", None, None, False
    shake_count, shake_stage, shake_start_x, shake_start_time, shake_completed = 0, "neutral", None, None, False
    stable_frame, stable_counter = None, 0
    gaze_start_time, selfie_taken = None, False
    fase_selfie = False

    print("🟡 Iniciando reto de vida...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
        raw_frame = frame.copy()
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb)

        if results.multi_face_landmarks:
            face_landmarks = results.multi_face_landmarks[0]
            mp_drawing.draw_landmarks(frame, face_landmarks, mp_face_mesh.FACEMESH_TESSELATION)

            ear_left = eye_aspect_ratio(face_landmarks.landmark, LEFT_EYE_FULL, frame.shape)
            ear_right = eye_aspect_ratio(face_landmarks.landmark, RIGHT_EYE_FULL, frame.shape)
            ear_avg = (ear_left + ear_right) / 2.0

            if not fase_selfie:
                # Parpadeo
                if not blink_completed:
                    if ear_avg < 0.26 and blink_ready:
                        if blink_counter == 0:
                            blink_start_time = time.time()
                        blink_counter += 1
                        blink_ready = False
                        print(f"✅ Parpadeo #{blink_counter}")
                    elif ear_avg >= 0.30:
                        blink_ready = True
                    if blink_counter > 0 and (time.time() - blink_start_time > 3):
                        print("⏱️ Parpadeos: tiempo excedido")
                        blink_counter, blink_start_time = 0, None
                    if blink_counter >= 3:
                        blink_completed = True
                        print("✅ Parpadeos completados")

                # Asentir ("sí")
                nose_y = face_landmarks.landmark[1].y
                if nod_start_y is None:
                    nod_start_y = nose_y
                    nod_start_time = time.time()
                delta_y = nose_y - nod_start_y
                if not nod_completed:
                    if nod_stage == "neutral" and delta_y > 0.03:
                        nod_stage = "down"
                    elif nod_stage == "down" and delta_y < -0.03:
                        nod_stage = "up"
                        nod_count += 1
                        print(f"✅ Asentimiento #{nod_count}")
                        nod_stage = "neutral"
                    if time.time() - nod_start_time > 5 and nod_count < 2:
                        print("⏱️ Asentir: tiempo excedido")
                        nod_count, nod_start_time = 0, time.time()
                    if nod_count >= 2:
                        nod_completed = True
                        print("✅ Asentir completado")

                # Negar ("no")
                nose_x = face_landmarks.landmark[1].x
                if shake_start_x is None:
                    shake_start_x = nose_x
                    shake_start_time = time.time()
                delta_x = nose_x - shake_start_x
                if not shake_completed:
                    if shake_stage == "neutral" and delta_x > 0.03:
                        shake_stage = "right"
                    elif shake_stage == "right" and delta_x < -0.03:
                        shake_stage = "left"
                        shake_count += 1
                        print(f"✅ Negación #{shake_count}")
                        shake_stage = "neutral"
                    if time.time() - shake_start_time > 5 and shake_count < 2:
                        print("⏱️ Negar: tiempo excedido")
                        shake_count, shake_start_time = 0, time.time()
                    if shake_count >= 2:
                        shake_completed = True
                        print("✅ Negar completado")

                # Mostrar progreso
                cv2.putText(frame, f"Parpadeos: {blink_counter}/3", (30, 40),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                cv2.putText(frame, f"Asentir: {nod_count}/2", (30, 70),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
                cv2.putText(frame, f"Negar: {shake_count}/2", (30, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)

                if blink_completed and nod_completed and shake_completed:
                    print("📸 Reto de vida completado. Prepara tu selfie...")
                    fase_selfie = True
                    start_selfie_time = time.time()
                    continue

            if fase_selfie and not selfie_taken:
                nose_x = face_landmarks.landmark[1].x
                nose_y = face_landmarks.landmark[1].y
                centered = 0.4 < nose_x < 0.6 and 0.4 < nose_y < 0.6
                eyes_open = ear_left > 0.26 and ear_right > 0.26

                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                gray = cv2.GaussianBlur(gray, (21, 21), 0)
                if stable_frame is None:
                    stable_frame = gray
                    continue
                diff = cv2.absdiff(stable_frame, gray)
                _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
                movement = cv2.countNonZero(thresh)
                stable = movement < 5000
                stable_counter = stable_counter + 1 if stable else 0

                looking_forward = is_looking_forward(face_landmarks.landmark)
                if looking_forward:
                    if gaze_start_time is None:
                        gaze_start_time = time.time()
                    gaze_duration = time.time() - gaze_start_time
                else:
                    gaze_start_time = None
                    gaze_duration = 0

                cv2.putText(frame, f"Cara centrada: {'✅' if centered else '❌'}", (30, 140),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 255, 200), 2)
                cv2.putText(frame, f"Ojos abiertos: {'✅' if eyes_open else '❌'}", (30, 170),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 255, 200), 2)
                cv2.putText(frame, f"Estable: {'✅' if stable_counter >= 5 else '❌'}", (30, 200),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 255, 200), 2)
                cv2.putText(frame, f"Mirando: {'✅' if gaze_duration >= 3 else f'{gaze_duration:.1f}s'}", (30, 230),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 255, 200), 2)

                if centered and eyes_open and stable_counter >= 5 and gaze_duration >= 3:
                    cv2.imwrite("selfie.jpg", raw_frame)
                    print("📷 Selfie capturada como 'selfie.jpg'")
                    selfie_taken = True
                    break
                stable_frame = gray

        cv2.imshow("Verificación Facial", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("🎬 Reto de vida completado correctamente.")
    if selfie_taken == True:
        return True
    else:
        return False



### Verificación de INE contra el CURP oficial

La función `verificar_ine_con_curp()` se encarga de validar que una persona que muestra su INE frente a la cámara sea realmente quien dice ser. Este proceso combina visión por computadora, OCR y automatización web para verificar la correspondencia entre los datos visibles en la INE y los registrados en el sitio oficial del gobierno mexicano ([https://www.gob.mx/curp/](https://www.gob.mx/curp/)).

---

#### Función interna `coincide_nombre_web_con_ocr()`

Esta función auxiliar compara si los datos extraídos del OCR (nombre y apellidos) coinciden con los datos oficiales consultados en línea.

```python
def coincide_nombre_web_con_ocr(nombre_web, ap1, ap2, lineas_ocr):
```

* Recibe como entrada:

  * Nombre y apellidos del sitio web oficial.
  * Líneas de texto obtenidas con OCR desde la imagen de la INE.
* Devuelve `True` si cada parte del nombre está presente en alguna línea OCR.
* Se usa para validar la coincidencia textual entre el documento físico y la fuente oficial.

---

#### Inicialización de componentes

```python
ocr = PaddleOCR(use_angle_cls=True, lang='es')
```

* Se inicializa el sistema OCR de PaddleOCR, configurado para idioma español y detección de orientación del texto.

```python
chrome_options = Options()
...
driver = webdriver.Chrome(options=chrome_options)
```

* Se configuran opciones para ejecutar Chrome en modo sin interfaz gráfica (headless).
* Se utiliza Selenium para automatizar la consulta al sitio del CURP.

---

#### Captura de la INE desde la cámara

La función inicia un bucle que:

* Muestra instrucciones al usuario para que coloque su INE frente a la cámara.
* Utiliza técnicas de detección de contornos con OpenCV para encontrar la tarjeta.
* Verifica que la tarjeta se mantenga visible y estable durante una cantidad determinada de frames (`required_frames = 150`).
* Una vez estable, guarda la imagen como `ine.jpg`.

---

#### Extracción de datos con OCR

Una vez que la INE es capturada:

```python
results = ocr.ocr("ine.jpg", cls=True)
```

* Se procesa la imagen capturada con OCR.
* Se extraen todas las líneas de texto reconocidas y se convierten a mayúsculas para comparación.
* Se buscan patrones específicos:

  * **CURP**: expresión regular para encontrar la cadena alfanumérica única.
  * **Fecha de nacimiento**: en formato dd/mm/aaaa o dd-mm-aaaa.

---

#### Consulta al sitio oficial del CURP

Usando Selenium:

1. Se accede al sitio web oficial.
2. Se introduce la CURP extraída.
3. Se recuperan los datos oficiales publicados (nombre, apellidos, fecha de nacimiento).
4. Se compara:

   * El nombre y apellidos con el OCR usando `coincide_nombre_web_con_ocr()`.
   * La fecha con la extraída del OCR.

---

#### Validación final

* Si tanto los nombres como la fecha coinciden, se considera que la persona fue verificada correctamente y se retorna `True`.
* Si no coinciden, se indica que hubo una discrepancia y se retorna `False`.
* Si ocurre un error en la conexión o carga de elementos del sitio, se captura la excepción, se limpia la sesión del navegador y se repite el proceso desde la captura de la INE.

---

#### Estructura de control

La función completa está envuelta en un `while True` para permitir reintentos automáticos en caso de error (por ejemplo, si la INE no fue detectada correctamente o hubo un fallo en la consulta web).

---

### Resultados

Esta función automatiza la validación de identidad documental utilizando una combinación de:

* Detección visual de tarjetas de identificación.
* Reconocimiento óptico de caracteres con PaddleOCR.
* Automatización web con Selenium.
* Verificación cruzada con fuentes oficiales.

Esto permite asegurar que el documento presentado pertenece realmente al usuario frente a la cámara y que no se trata de un intento de suplantación.


In [ ]:
def verificar_ine_con_curp():

    def coincide_nombre_web_con_ocr(nombre_web, ap1, ap2, lineas_ocr):
        partes = [nombre_web, ap1, ap2]
        for parte in partes:
            if not any(parte.upper() in linea for linea in lineas_ocr):
                return False
        return True

    ocr = PaddleOCR(use_angle_cls=True, lang='es')
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")

    while True:
        print("🪪 Muestra tu INE al centro de la cámara, bien enfocada y estable...")

        cap = cv2.VideoCapture(0)
        stable_frame = None
        ine_capturada = False
        stable_counter = 0
        required_frames = 150

        while cap.isOpened():
            ret, raw_frame = cap.read()
            if not ret:
                break

            frame = raw_frame.copy()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (5, 5), 0)
            edged = cv2.Canny(blur, 75, 200)
            cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)
            cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:5]

            found_card = False
            frame_area = frame.shape[0] * frame.shape[1]

            for c in cnts:
                approx = cv2.approxPolyDP(c, 0.02 * cv2.arcLength(c, True), True)
                area = cv2.contourArea(c)
                if len(approx) == 4 and area > 0.20 * frame_area:
                    cv2.drawContours(frame, [approx], -1, (0, 255, 0), 2)
                    stable_counter += 1
                    found_card = True
                    break

            if not found_card:
                stable_counter = 0

            porcentaje = int((stable_counter / required_frames) * 100)
            cv2.putText(frame, f"INE detectada: {porcentaje}%", (20, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
            cv2.putText(frame, "Mantén la INE visible y grande durante 5 segundos",
                        (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

            cv2.imshow("Detección de INE", frame)

            if stable_counter >= required_frames:
                cv2.imwrite("ine.jpg", raw_frame)
                print("📸 INE capturada como 'ine.jpg'")
                ine_capturada = True
                break

            if cv2.waitKey(1) & 0xFF == 27:
                cap.release()
                cv2.destroyAllWindows()
                return False

        cap.release()
        cv2.destroyAllWindows()

        if not ine_capturada:
            continue

        print("🔍 Procesando INE con OCR...")
        results = ocr.ocr("ine.jpg", cls=True)
        lineas_ocr_ine = [line[1][0].strip().upper() for line in results[0]]

        curp_match = next((re.search(r"\b[A-Z]{4}\d{6}[A-Z0-9]{8}\b", l)
                           for l in lineas_ocr_ine if re.search(r"\b[A-Z]{4}\d{6}[A-Z0-9]{8}\b", l)), None)
        curp = curp_match.group() if curp_match else "No detectado"

        fecha_match = next((re.search(r"\b\d{2}[/-]\d{2}[/-]\d{4}\b", l)
                            for l in lineas_ocr_ine if re.search(r"\b\d{2}[/-]\d{2}[/-]\d{4}\b", l)), None)
        fecha = fecha_match.group() if fecha_match else "No detectada"

        print("📌 CURP detectado:", curp)
        print("📌 Fecha de nacimiento:", fecha)

        print("🌐 Consultando datos oficiales del CURP...")
        try:
            driver = webdriver.Chrome(options=chrome_options)
            driver.get("https://www.gob.mx/curp/")

            input_curp = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "curpinput"))
            )
            input_curp.send_keys(curp)
            boton_buscar = driver.find_element(By.ID, "searchButton")
            boton_buscar.click()

            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//td[@style='text-transform: uppercase;']"))
            )

            celdas = driver.find_elements(By.XPATH, "//td[@style='text-transform: uppercase;']")
            if len(celdas) >= 7:
                nombre_web = celdas[1].text.strip()
                apellido1_web = celdas[2].text.strip()
                apellido2_web = celdas[3].text.strip()
                fecha_web = celdas[5].text.strip()

                print("📄 Datos oficiales:", apellido1_web, apellido2_web, nombre_web, fecha_web)
                nombre_ok = coincide_nombre_web_con_ocr(nombre_web, apellido1_web, apellido2_web, lineas_ocr_ine)
                fecha_ok = (fecha.strip() == fecha_web.strip())

                if nombre_ok and fecha_ok:
                    print("✅ Identidad confirmada con datos oficiales.")
                    driver.quit()
                    return True
                else:
                    print("❌ Los datos de la INE no coinciden con el CURP.")
            else:
                print("❌ No se extrajeron suficientes datos del sitio.")
            driver.quit()
            time.sleep(2)

        except (WebDriverException, TimeoutException) as e:
            print(f"⚠️ Error en Selenium: {e}")
            print("🔁 Ocurrió un error al consultar el CURP. Volveremos a escanear la INE...")
            try:
                driver.quit()
            except:
                pass
            time.sleep(2)
            continue  # vuelve al loop a escanear de nuevo




### Comparación de rostro entre INE y selfie

La función `comparar_rostros_ine_selfie()` tiene como propósito verificar que el rostro detectado en la INE coincida con el rostro de la selfie capturada previamente durante el reto de vida. Esta verificación biométrica fortalece el sistema de autenticación asegurando que el documento pertenece a la persona que lo presenta.

---

#### Paso 1: Detección del rostro en la INE

```python
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.6)
```

* Se inicializa el detector facial de MediaPipe.
* `model_selection=1` indica que se utilizará el modelo más preciso para imágenes cercanas.
* `min_detection_confidence=0.6` establece el umbral mínimo de confianza para considerar una detección válida.

```python
image = cv2.imread("ine.jpg")
```

* Se carga la imagen previamente guardada de la INE.
* Si no existe, se detiene el proceso y se retorna `False`.

```python
results = face_detection.process(image_rgb)
```

* La imagen es procesada para encontrar rostros.
* Si se detecta al menos un rostro, se selecciona el de mayor área (más grande), suponiendo que es el rostro principal.

```python
cropped_face = image[y:y + h_box, x:x + w_box]
cv2.imwrite("ine_face.jpg", cropped_face)
```

* Se recorta y guarda el rostro detectado como `ine_face.jpg` para la posterior comparación.

---

#### Paso 2: Comparación facial con la selfie

```python
if not os.path.exists("selfie.jpg") or not os.path.exists("ine_face.jpg"):
```

* Se valida la existencia de ambas imágenes necesarias para la comparación.

```python
img1 = face_recognition.load_image_file("selfie.jpg")
img2 = face_recognition.load_image_file("ine_face.jpg")
```

* Ambas imágenes son cargadas con la librería `face_recognition`.

```python
enc1 = face_recognition.face_encodings(img1)
enc2 = face_recognition.face_encodings(img2)
```

* Se generan los vectores de características faciales (face embeddings) a partir de cada imagen.

---

#### Evaluación de coincidencia

```python
result = face_recognition.compare_faces([enc1[0]], enc2[0])
distance = face_recognition.face_distance([enc1[0]], enc2[0])[0]
```

* Se compara la similitud entre los vectores faciales mediante:

  * `compare_faces()`: entrega `True` si los rostros coinciden.
  * `face_distance()`: devuelve la distancia euclidiana entre embeddings; cuanto menor sea, más parecidos son.

**Resultado:**

* Si los vectores coinciden (`result[0] == True`), se confirma la correspondencia facial y se retorna `True`.
* Si no coinciden o si los vectores no pudieron generarse, se considera que no hay coincidencia y se retorna `False`.

---

Esta función permite validar biométricamente si la persona que aparece en la INE es la misma que realizó el reto de vida, fortaleciendo el sistema de verificación de identidad contra intentos de suplantación. Se basa en detección de rostro (MediaPipe) y comparación de características faciales (face\_recognition) para lograr una evaluación confiable.


In [ ]:
def comparar_rostros_ine_selfie():

    print("🧠 Detectando rostro en la INE...")

    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.6)

    image = cv2.imread("ine.jpg")
    if image is None:
        print("❌ No se encontró 'ine.jpg'.")
        return False

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_detection.process(image_rgb)

    if results.detections:
        h, w, _ = image.shape
        max_area = 0
        best_face = None

        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box
            x, y = int(bbox.xmin * w), int(bbox.ymin * h)
            w_box, h_box = int(bbox.width * w), int(bbox.height * h)
            area = w_box * h_box

            if area > max_area:
                max_area = area
                best_face = (x, y, w_box, h_box)

        if best_face:
            x, y, w_box, h_box = best_face
            x, y = max(0, x), max(0, y)
            cropped_face = image[y:y + h_box, x:x + w_box]
            cv2.imwrite("ine_face.jpg", cropped_face)
            print("✅ Rostro recortado y guardado como 'ine_face.jpg'")
    else:
        print("❌ No se detectó rostro en la INE.")
        return False

    # Comparación facial
    print("🧪 Comparando rostro de INE con selfie...")

    if not os.path.exists("selfie.jpg") or not os.path.exists("ine_face.jpg"):
        print("❌ Faltan imágenes para la comparación facial.")
        return False

    img1 = face_recognition.load_image_file("selfie.jpg")
    img2 = face_recognition.load_image_file("ine_face.jpg")

    enc1 = face_recognition.face_encodings(img1)
    enc2 = face_recognition.face_encodings(img2)

    if enc1 and enc2:
        result = face_recognition.compare_faces([enc1[0]], enc2[0])
        distance = face_recognition.face_distance([enc1[0]], enc2[0])[0]

        if result[0]:
            print(f"✅ Rostros coinciden (distancia: {distance:.4f})")
            return True
        else:
            print(f"❌ Rostros NO coinciden (distancia: {distance:.4f})")
            return False
    else:
        print("❌ No se pudieron codificar ambos rostros correctamente.")
        return False



### Predicción de Deepfake sobre un video capturado

La función `predecir_deepfake()` evalúa si un video contiene contenido manipulado (deepfake) utilizando un modelo previamente entrenado con secuencias de rostro y vectores de puntos faciales. Esta función es la última etapa de verificación del sistema y actúa como filtro biométrico basado en redes neuronales.

---

#### Definición del modelo: `DeepfakeDetector`

```python
class DeepfakeDetector(nn.Module):
```

Este modelo combina:

* **EfficientNet-B0** (preentrenado): como extractor de características visuales (embeddings de imágenes faciales).
* **LSTM bidireccional**: para capturar dependencias temporales entre los frames de un video.
* **Red fully-connected**: que toma la última salida del LSTM y produce una probabilidad de deepfake.

**Entradas del modelo:**

* Secuencia de imágenes faciales (`x_imgs`)
* Vectores de landmarks por frame (`x_lmks`)

**Salida:**

* Valor entre 0 y 1 indicando la probabilidad de que el video sea un deepfake.

---

#### Preprocesamiento del video

```python
cap = cv2.VideoCapture(video_path)
```

* El video de entrada es leído con OpenCV.
* Se seleccionan 25 frames distribuidos equitativamente a lo largo del video.
* Se procesan hasta reunir 16 frames válidos con rostro detectado.

---

#### Extracción de rostros y landmarks

##### Función `crop_face_from_landmarks()`

* Recorta el rostro detectado en cada frame, utilizando los puntos de referencia proporcionados por MediaPipe.
* Se añade un margen alrededor del rostro para asegurar que esté completo.
* Las imágenes se redimensionan a 256×256 píxeles.

##### Función `extract_landmark_vector()`

* Extrae un vector de 5 valores normalizados por frame:

  * Coordenadas horizontales de los ojos.
  * Coordenadas verticales de nariz y extremos de boca.
* Este vector representa una firma geométrica básica del rostro y se usa junto al embedding de imagen.

---

#### Almacenamiento de evidencia

La primera imagen facial recortada del video se guarda como `"evidencia.jpg"` para auditoría o respaldo visual del análisis.

---

#### Verificación de frames

```python
if len(images) < sequence_length:
    return None
```

* Si no se reúnen al menos 16 frames válidos, se cancela la predicción por falta de evidencia.

---

#### Carga del modelo y predicción

```python
model.load_state_dict(torch.load(model_path))
model.eval()
```

* Se carga el modelo entrenado desde un archivo `.pth`.
* Se colocan las entradas en el dispositivo correspondiente (`cuda` o `cpu`).
* Se realiza la inferencia con el modelo sin actualizar gradientes (`torch.no_grad()`).

---

#### Resultado final

```python
label = "FAKE" if prob > 0.5 else "REAL"
```

* Si la probabilidad es mayor a 0.5, el video se clasifica como deepfake.
* Si es menor o igual, se considera real.
* Se devuelve un diccionario con:

  * Etiqueta (`label`)
  * Probabilidad numérica (`prob`)
  * Ruta de la imagen de evidencia (`evidencia.jpg`)

---


Esta función permite realizar un análisis automático sobre la autenticidad de un video capturado, combinando visión computacional, landmarks faciales y un modelo de detección entrenado. Sirve como filtro final para confirmar que la persona que superó el reto de vida y mostró su INE no está siendo suplantada mediante técnicas de manipulación audiovisual.


In [ ]:
def predecir_deepfake(video_path, model_path="mediapipe_model.pth"):
    
    class DeepfakeDetector(nn.Module):
        def __init__(self):
            super().__init__()
            self.cnn = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
            self.cnn.classifier = nn.Identity()
            self.embedding_dim = 1280
            self.sequence_length = 16

            self.lstm = nn.LSTM(input_size=1285, hidden_size=128, num_layers=1,
                                batch_first=True, bidirectional=True)
            self.fc = nn.Sequential(
                nn.Linear(256, 64),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(64, 1),
                nn.Sigmoid()
            )

        def forward(self, x_imgs, x_lmks):
            B, T, C, H, W = x_imgs.shape
            x_imgs = x_imgs.view(B * T, C, H, W)
            features = self.cnn(x_imgs)
            features = features.view(B, T, -1)
            combined = torch.cat([features, x_lmks], dim=2)
            out, _ = self.lstm(combined)
            out = out[:, -1, :]
            return self.fc(out).squeeze(1)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    sequence_length = 16
    candidate_frames = 25
    image_size = (256, 256)
    transform = transforms.Compose([transforms.ToTensor()])
    mp_face_mesh = mp.solutions.face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1,
                                                   refine_landmarks=True, min_detection_confidence=0.5)

    def extract_landmark_vector(landmarks, frame_shape):
        h, w, _ = frame_shape
        def norm(x): return x / w
        def norm_y(y): return y / h
        left_eye = landmarks.landmark[33]
        right_eye = landmarks.landmark[263]
        nose = landmarks.landmark[1]
        mouth_left = landmarks.landmark[61]
        mouth_right = landmarks.landmark[291]
        return np.array([
            norm(left_eye.x), norm(right_eye.x),
            norm_y(nose.y),
            norm_y(mouth_left.y),
            norm_y(mouth_right.y)
        ], dtype=np.float32)

    def crop_face_from_landmarks(landmarks, frame):
        h, w, _ = frame.shape
        x_coords = [lm.x for lm in landmarks.landmark]
        y_coords = [lm.y for lm in landmarks.landmark]
        min_x, max_x = int(min(x_coords) * w), int(max(x_coords) * w)
        min_y, max_y = int(min(y_coords) * h), int(max(y_coords) * h)
        margin_x = int((max_x - min_x) * 0.2)
        margin_y = int((max_y - min_y) * 0.2)
        x1 = max(min_x - margin_x, 0)
        y1 = max(min_y - margin_y, 0)
        x2 = min(max_x + margin_x, w)
        y2 = min(max_y + margin_y, h)
        face_crop = frame[y1:y2, x1:x2]
        return cv2.resize(face_crop, image_size)

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, candidate_frames).astype(int)

    images, landmarks_list = [], []
    evidencia_guardada = False

    for idx in frame_indices:
        if len(images) >= sequence_length:
            break
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            continue
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = mp_face_mesh.process(rgb)
        if results.multi_face_landmarks:
            try:
                lmks = results.multi_face_landmarks[0]
                cropped = crop_face_from_landmarks(lmks, frame)
                lmk_vector = extract_landmark_vector(lmks, frame.shape)

                if not evidencia_guardada:
                    cv2.imwrite("evidencia.jpg", cropped)
                    evidencia_guardada = True

                images.append(transform(cropped))
                landmarks_list.append(torch.tensor(lmk_vector, dtype=torch.float32))
            except:
                continue

    cap.release()
    mp_face_mesh.close()

    if len(images) < sequence_length:
        print(f"⚠️ Solo se obtuvieron {len(images)} frames válidos. No se puede hacer inferencia.")
        return None

    x_imgs = torch.stack(images[:sequence_length]).unsqueeze(0).to(device)
    x_lmks = torch.stack(landmarks_list[:sequence_length]).unsqueeze(0).to(device)

    model = DeepfakeDetector().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    with torch.no_grad():
        output = model(x_imgs, x_lmks)
        prob = output.item()
        label = "FAKE" if prob > 0.5 else "REAL"
        print(f"\n🧪 Resultado: {label}  |  Probabilidad: {prob:.4f} | Evidencia: evidencia.jpg")
        return {"label": label, "prob": prob, "evidencia": "evidencia.jpg"}


Aquí tienes la explicación en formato Markdown, sin emojis, que describe cómo se utilizan las funciones previamente definidas en la ejecución final del sistema:

---

## Flujo de ejecución del sistema de verificación de identidad

El siguiente bloque de código ejecuta de manera secuencial todo el proceso de validación de identidad. Cada función corresponde a una etapa crítica en el sistema y debe completarse correctamente para continuar con la siguiente.

---

### Orden y propósito de cada función

1. **`realizar_reto_de_vida()`**

   * Verifica que el usuario esté presente en tiempo real realizando gestos naturales (parpadeo, asentir y negar).
   * Si se completa con éxito, captura una selfie (`selfie.jpg`) y guarda un video (`verificacion_video.mp4`).

2. **`verificar_ine_con_curp()`**

   * Detecta y captura una imagen clara de la INE desde la cámara.
   * Extrae datos mediante OCR y los compara con los datos oficiales consultados en el sitio del CURP.
   * Solo si los nombres y la fecha de nacimiento coinciden, se considera válida la identidad documental.

3. **`comparar_rostros_ine_selfie()`**

   * Detecta el rostro en la INE y lo compara con la selfie tomada anteriormente.
   * Utiliza reconocimiento facial para confirmar que ambos rostros pertenecen a la misma persona.

4. **`predecir_deepfake()`**

   * Evalúa el video completo capturado durante el reto de vida para verificar si contiene señales de manipulación (deepfake).
   * Devuelve una predicción final (`FAKE` o `REAL`) con la probabilidad asociada.

---

### Resultado final

Si todas las etapas se completan exitosamente, se imprime el resultado final indicando si se trata de una persona real o no, basado en la predicción del modelo de detección de deepfakes.

En caso de que alguna de las etapas falle (ya sea por errores de lectura, detección facial, discrepancia de datos o señales de manipulación), se cancela el flujo y se informa que no fue posible verificar la identidad de manera confiable.

---

Este flujo garantiza una validación robusta, combinando detección de actividad real, verificación documental oficial, comparación biométrica y análisis de manipulación digital.


In [26]:
if realizar_reto_de_vida() == True:
    if verificar_ine_con_curp() == True:
        if comparar_rostros_ine_selfie() == True:
            resultado = predecir_deepfake("verificacion_video.mp4", "mediapipe_model.pth")
            if resultado:
                print("Veredicto final: Es una persona", resultado["label"])
            else:
                print("❌ No se pudo analizar el video.")
        else:
            print("No se pudo comprobar que se tratara de una persona real")
    else:
        print("No se pudo comprobar que se tratara de una persona real")
else:
    print("No se pudo comprobar que se tratara de una persona real")

🟡 Iniciando reto de vida...
✅ Parpadeo #1
✅ Parpadeo #2
✅ Negación #1
✅ Parpadeo #3
✅ Parpadeos completados
⏱️ Asentir: tiempo excedido
⏱️ Negar: tiempo excedido
✅ Asentimiento #1
✅ Asentimiento #2
✅ Asentir completado
✅ Negación #1
⏱️ Negar: tiempo excedido
✅ Negación #1
✅ Negación #2
✅ Negar completado
📸 Reto de vida completado. Prepara tu selfie...
📷 Selfie capturada como 'selfie.jpg'
🎬 Reto de vida completado correctamente.
[2025/05/07 22:43:15] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast'

Resultado:

```
🟡 Iniciando reto de vida...
✅ Parpadeo #1
✅ Parpadeo #2
✅ Negación #1
✅ Parpadeo #3
✅ Parpadeos completados
⏱️ Asentir: tiempo excedido
⏱️ Negar: tiempo excedido
✅ Asentimiento #1
✅ Asentimiento #2
✅ Asentir completado
✅ Negación #1
⏱️ Negar: tiempo excedido
✅ Negación #1
✅ Negación #2
✅ Negar completado
📸 Reto de vida completado. Prepara tu selfie...
📷 Selfie capturada como 'selfie.jpg'
🎬 Reto de vida completado correctamente.
[2025/05/07 22:43:15] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\rec\\latin\\latin_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='c:\\Users\\Hermanos\\Desktop\\Proyecto Deepfake\\.venv-mediapipe\\lib\\site-packages\\paddleocr\\ppocr\\utils\\dict\\latin_dict.txt', use_space_char=True, vis_font_path='./doc/fonts/simfang.ttf', drop_score=0.5, e2e_algorithm='PGNet', e2e_model_dir=None, e2e_limit_side_len=768, e2e_limit_type='max', e2e_pgnet_score_thresh=0.5, e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_pgnet_valid_set='totaltext', e2e_pgnet_mode='fast', use_angle_cls=True, cls_model_dir='C:\\Users\\Hermanos/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_image_shape='3, 48, 192', label_list=['0', '180'], cls_batch_num=6, cls_thresh=0.9, enable_mkldnn=False, cpu_threads=10, use_pdserving=False, warmup=False, sr_model_dir=None, sr_image_shape='3, 32, 128', sr_batch_num=1, draw_img_save_dir='./inference_results', save_crop_res=False, crop_res_save_dir='./output', use_mp=False, total_process_num=1, process_id=0, benchmark=False, save_log_path='./log_output/', show_log=True, use_onnx=False, onnx_providers=False, onnx_sess_options=False, return_word_box=False, output='./output', table_max_len=488, table_algorithm='TableAttn', table_model_dir=None, merge_no_span_structure=True, table_char_dict_path=None, formula_algorithm='LaTeXOCR', formula_model_dir=None, formula_char_dict_path=None, formula_batch_num=1, layout_model_dir=None, layout_dict_path=None, layout_score_threshold=0.5, layout_nms_threshold=0.5, kie_algorithm='LayoutXLM', ser_model_dir=None, re_model_dir=None, use_visual_backbone=True, ser_dict_path='../train_data/XFUND/class_list_xfun.txt', ocr_order_method=None, mode='structure', image_orientation=False, layout=True, table=True, formula=False, ocr=True, recovery=False, recovery_to_markdown=False, use_pdf2docx_api=False, invert=False, binarize=False, alphacolor=(255, 255, 255), lang='es', det=True, rec=True, type='ocr', savefile=False, ocr_version='PP-OCRv4', structure_version='PP-StructureV2')
🪪 Muestra tu INE al centro de la cámara, bien enfocada y estable...
📸 INE capturada como 'ine.jpg'
🔍 Procesando INE con OCR...
[2025/05/07 22:44:15] ppocr DEBUG: dt_boxes num : 25, elapsed : 0.04928088188171387
[2025/05/07 22:44:15] ppocr DEBUG: cls num  : 25, elapsed : 0.07489395141601562
[2025/05/07 22:44:16] ppocr DEBUG: rec_res num  : 25, elapsed : 0.33364295959472656
📌 CURP detectado: ********
📌 Fecha de nacimiento: ********
🌐 Consultando datos oficiales del CURP...
📄 Datos oficiales: FLORES MENDOZA JOSUE EMMANUEL ********
✅ Identidad confirmada con datos oficiales.
🧠 Detectando rostro en la INE...
✅ Rostro recortado y guardado como 'ine_face.jpg'
🧪 Comparando rostro de INE con selfie...
✅ Rostros coinciden (distancia: 0.4384)

🧪 Resultado: REAL  |  Probabilidad: 0.3616 | Evidencia: evidencia.jpg
Veredicto final: Es una persona REAL